In [18]:
from openai import OpenAI, AsyncOpenAI
from getpass import getpass

import nest_asyncio

import random
import datetime as dt
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel, AsyncOpenAI
from pydantic_ai.providers.openai import OpenAIProvider

from dotenv import load_dotenv
import os

In [19]:
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")

In [20]:
client = AsyncOpenAI(
    base_url="https://api.openai.com/v1",
    api_key=API_KEY,
)

MODEL_ID = "gpt-4.1"

In [21]:

nest_asyncio.apply()

In [22]:
model = OpenAIModel(
    MODEL_ID,
    provider=OpenAIProvider(
        openai_client=client,

    )
)
agent = Agent(
    model=model,
    deps_type=str,
    system_prompt=(
"""
        You're a dice game, you should roll the die and see if the number
        you get back matches the user's guess. If so, tell them they're a winner.
        Use the player's name in the response.
""".strip()
    ),
)


@agent.tool_plain()
def roll_dice() -> str:
    """Roll a six-sided die and return the result. No arguments should be passed to this tool!
    """
    return str(random.randint(1, 6))

@agent.tool  
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps

@agent.tool_plain()
def date() -> str:
    """Returns current date in format YYYY-MM-DD. No arguments should be passed to this tool!
    """
    return dt.datetime.now().strftime("%Y-%m-%d")

# Example Calls:
# dice_result = agent.run_sync('My guess is: 4', model_settings={'temperature': 0.8})
# dice_result = agent.run_sync('What day is it today?', model_settings={'temperature': 0.8})
# dice_result = agent.run_sync(
#     'Answer the question: What day is it today? and draw a random number. My guess is: 4',
#     model_settings={'temperature': 0.8}
# )

dice_result = agent.run_sync(
    'Answer the question: What day is it today? Then draw a random number. My guess is: 4.',
    deps='John',
    model_settings={'temperature': 0.8}
)


print(dice_result.output)

/var/folders/y7/8fvphx1s0b53_2_bzrh3s81m0000gn/T/ipykernel_69749/2191329574.py:1: DeprecationWarning: `OpenAIModel` was renamed to `OpenAIChatModel` to clearly distinguish it from `OpenAIResponsesModel` which uses OpenAI's newer Responses API. Use that unless you're using an OpenAI Chat Completions-compatible API, or require a feature that the Responses API doesn't support yet like audio.
  model = OpenAIModel(


Today is 2026-02-05.

Now, let's see your guess! You guessed 4, but the die rolled a 6. Sorry John, you didn't win this time. Try again!


In [23]:
for mr in dice_result.all_messages():
    print(mr)
    print()

ModelRequest(parts=[SystemPromptPart(content="You're a dice game, you should roll the die and see if the number\n        you get back matches the user's guess. If so, tell them they're a winner.\n        Use the player's name in the response.", timestamp=datetime.datetime(2026, 2, 5, 9, 51, 17, 506606, tzinfo=datetime.timezone.utc)), UserPromptPart(content='Answer the question: What day is it today? Then draw a random number. My guess is: 4.', timestamp=datetime.datetime(2026, 2, 5, 9, 51, 17, 506631, tzinfo=datetime.timezone.utc))], timestamp=datetime.datetime(2026, 2, 5, 9, 51, 17, 506781, tzinfo=datetime.timezone.utc), run_id='15f404df-68c6-4fc6-b0cb-82f7417d0b1c')

ModelResponse(parts=[ToolCallPart(tool_name='date', args='{}', tool_call_id='call_7Cn3CXLxWKf3OCmQh2Pm0oMI'), ToolCallPart(tool_name='roll_dice', args='{}', tool_call_id='call_X9F9sUFk4pZsT4rBoij7w0rD')], usage=RequestUsage(input_tokens=156, output_tokens=39, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, '